In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score,\
        cross_validate, ShuffleSplit, GridSearchCV
from sklearn.impute import MissingIndicator, SimpleImputer 
from sklearn.metrics import roc_auc_score, plot_roc_curve, accuracy_score,\
        precision_score, recall_score, log_loss, plot_confusion_matrix, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

In [2]:
pumps_X = pd.read_csv('..\Data\Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv')

In [3]:
pumps_y = pd.read_csv('..\Data\Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv')

In [4]:
pumps_X.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [5]:
pumps_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [6]:
pumps_X.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [7]:
pumps_y.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [8]:
pumps = pd.merge(pumps_y, pumps_X, how='left')

In [9]:
pumps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   status_group           59400 non-null  object 
 2   amount_tsh             59400 non-null  float64
 3   date_recorded          59400 non-null  object 
 4   funder                 55765 non-null  object 
 5   gps_height             59400 non-null  int64  
 6   installer              55745 non-null  object 
 7   longitude              59400 non-null  float64
 8   latitude               59400 non-null  float64
 9   wpt_name               59400 non-null  object 
 10  num_private            59400 non-null  int64  
 11  basin                  59400 non-null  object 
 12  subvillage             59029 non-null  object 
 13  region                 59400 non-null  object 
 14  region_code            59400 non-null  int64  
 15  di

In [10]:
drop_columns = ['id', 'extraction_type', 'extraction_type_group', 'scheme_name', 'payment',\
                'quality_group', 'quantity_group', 'source_type', 'waterpoint_type_group',\
                'region_code','district_code', 'ward', 'subvillage', 'lga', 'num_private',\
                'recorded_by']

In [11]:
pumps_trim = pumps.drop(columns=drop_columns, axis=1)

In [12]:
pumps_trim.head()

,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,...,construction_year,extraction_type_class,management,management_group,payment_type,water_quality,quantity,source,source_class,waterpoint_type
0,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,Lake Nyasa,...,1999,gravity,vwc,user-group,annually,soft,enough,spring,groundwater,communal standpipe
1,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,Lake Victoria,...,2010,gravity,wug,user-group,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe
2,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,Pangani,...,2009,gravity,vwc,user-group,per bucket,soft,enough,dam,surface,communal standpipe multiple
3,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,...,1986,submersible,vwc,user-group,never pay,soft,dry,machine dbh,groundwater,communal standpipe multiple
4,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,Lake Victoria,...,0,gravity,other,other,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe


In [13]:
pumps_trim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   status_group           59400 non-null  object 
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   basin                  59400 non-null  object 
 10  region                 59400 non-null  object 
 11  population             59400 non-null  int64  
 12  public_meeting         56066 non-null  object 
 13  scheme_management      55523 non-null  object 
 14  permit                 56344 non-null  object 
 15  co

In [14]:
pt = pumps_trim.copy()

In [15]:
pt['funder'].fillna('unknown', inplace=True)

In [16]:
pt['installer'].fillna('unknown', inplace=True)

In [17]:
pt['public_meeting'].fillna('unknown', inplace=True)

In [18]:
pt['scheme_management'].fillna('unknown', inplace=True)

In [19]:
pt['permit'].fillna('unknown', inplace=True)

In [20]:
pt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   status_group           59400 non-null  object 
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 59400 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              59400 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   basin                  59400 non-null  object 
 10  region                 59400 non-null  object 
 11  population             59400 non-null  int64  
 12  public_meeting         59400 non-null  object 
 13  scheme_management      59400 non-null  object 
 14  permit                 59400 non-null  object 
 15  co

In [21]:
X = pt.drop('status_group', axis=1)

In [22]:
y = pt['status_group']

In [23]:
X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=1979)

In [24]:
y_replace = {'functional':0, 'functional needs repair':1, 'non functional':2}

In [25]:
y_train = y_train.replace(y_replace)

In [26]:
lat_lon = {
    'Iringa' : [-7.773094,35.699120],
    'Shinyanga' : [-3.680996,33.427139],
    'Mbeya' : [-8.909401,33.460773],
    'Kilimanjaro' : [-3.393146,37.550789],
    'Morogoro' : [-6.830373,37.670589],
    'Arusha' : [-3.386925,36.682995],
    'Kagera' : [-1.3001,31.2626],
    'Mwanza' : [-2.5164,32.9175],
    'Kigoma' : [-4.8824,29.6615],
    'Pwani' : [-7.3238,38.8205],
    'Ruvuma' : [-10.6879,36.2631],
    'Tanga' : [-5.0889,39.1023],
    'Dodoma' : [-6.161184,35.745426],
    'Singida' : [-4.8254,34.7630],
    'Tabora' : [-5.030461,32.819431],
    'Mara' : [-1.7754,34.1532],
    'Rukwa' : [-8.0109,31.4456],
    'Mtwara' : [-10.3112,40.1760],
    'Manyara' : [-4.3150,36.9541],
    'Lindi' : [-9.9876,39.6982],
    'Dar es Salaam' : [-6.776012,39.178326]
}

In [42]:
def fix_lat_long(df):
    longs = []
    for long in df:
        long.append
        print(long)
        if long['longitude'] == 0:
            lat = lat_lon[frame_line['region'][0]]
            lon = lat_lon[frame_line['region'][1]]
            df['longitude'] = lon
            df['latitude'] = lat
    return frame_line

In [43]:
from sklearn.preprocessing import FunctionTransformer

In [44]:
ft = FunctionTransformer(fix_lat_long)

In [45]:
ft.fit(X_train)

FunctionTransformer(func=<function fix_lat_long at 0x000001DE74933310>)

In [46]:
lat_lon_fixed_X_train = ft.transform(X_train)

amount_tsh


TypeError: string indices must be integers